In [229]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)

from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RefineDocumentsChain, LLMChain

class SummarizerAgent:
    def __init__(
        self,
        llm_model_name,
        summary_prompt,
        refine_prompt,
        vector_store,
        kb_metadata=None,
        document_location=None,
    ):
        self.llm_model_name = llm_model_name
        self.summary_prompt = summary_prompt
        self.refine_prompt = refine_prompt
        self.vector_store = vector_store
        self.kb_metadata = kb_metadata
        self.document_location = document_location

        self.create_index()

    def create_index(self):
        if self.document_location is None:
            return

        reader = SimpleDirectoryReader(input_dir=self.document_location)
        documents = reader.load_data()
        index_loaded = False
        storage_context = None
        try:
            storage_context = StorageContext.from_defaults(vector_store=self.vector_store)
            self.data_index = load_index_from_storage(storage_context)
            index_loaded = True
        except Exception as e:
            print(f"Failed to load index from storage: {e}")
            index_loaded = False
        if not index_loaded:
            self.data_index = VectorStoreIndex.from_documents(
                documents, storage_context=storage_context, kb_metadata=self.kb_metadata
            )

    def retrieve_relevant_documents(self, query):
        if self.document_location is None:
            return []

        retriever = self.data_index.as_retriever(verbose=True)
        search_results = retriever.retrieve(query)
        return "\n".join(search_results)

    def summarize(self, text_to_summarize):
        try:
            llm = ChatOpenAI(model=self.llm_model_name)
            text_splitter = RecursiveCharacterTextSplitter(
                separators=["\n\n", "\n"," "], chunk_size=3000, chunk_overlap=500
            )
            docs = text_splitter.create_documents([text_to_summarize])

            initial_prompt_template = PromptTemplate(
                template=self.summary_prompt, input_variables=["text"]
            )

            refine_prompt_template = PromptTemplate(
                template=self.refine_prompt, input_variables=["existing_summary"]
            )

            refine_chain = RefineDocumentsChain(
                initial_llm_chain=LLMChain(llm=llm, prompt=initial_prompt_template),
                refine_llm_chain=LLMChain(llm=llm, prompt=refine_prompt_template),
                document_variable_name="text",
                initial_response_name="existing_summary",
            )

            summary = refine_chain.run(docs)
            return summary
        except Exception as e:
            print(f"An error occurred during summarization: {e}")
            print("Failed to summarize")
            return None

import json

kb_metadata = {
    "name": "HPI summary example 1",
    "data_type": "text",
    "metadata": {"category": "AI", "description": "Knowledge base on AI topics"},
    "status": "active"
}


In [ ]:
#hpi

In [230]:

summary_prompt = """
The following is a clinical conversation between a doctor and a patient. Generate a detailed History of Present Illness (HPI) for the patient. The HPI should include the following sections only:

1. Chief Complaint:
2. Previous Occurrences:
3. Family History:
4. Symptoms:

Conversation Transcript:
{text}

Only include the above sections as mentioned in the transcript. Don't include additional details like physical exam findings or assessment plan.
"""

refine_prompt = """
Based on the provided clinical conversation and the following transcript, generate a detailed History of Present Illness (HPI) for the patient. The HPI should include the following sections:

1. Chief Complaint: Brief description of only the primary symptom.
2. Previous Occurrences: Any similar past episodes.
3. Family History: Relevant family medical history.
4. Symptoms: Only the current symptoms.

Current HPI: {existing_summary}

Only include the above sections and no other information.
"""

summary_agent = SummarizerAgent(
    llm_model_name="gpt-4o",
    summary_prompt=summary_prompt,
    refine_prompt=refine_prompt,
    vector_store=None,
    kb_metadata=kb_metadata,
    document_location=None
)

with open('./meva/June12-01_AbdominalPain.json', 'r') as file:
    transcript_data = json.load(file)

summary_result = summary_agent.summarize(transcript_data['results']['transcripts'][0]['transcript'])
print(summary_result)


### History of Present Illness (HPI)

#### 1. Chief Complaint:
The patient reports experiencing sharp abdominal pain located right underneath her right ribs.

#### 2. Previous Occurrences:
The patient is currently nine weeks pregnant and has had four prior pregnancies. She reports one miscarriage but no other complications with her previous pregnancies. This is her fifth pregnancy.

#### 3. Family History:
The patient did not provide specific details about family history in the transcript.

#### 4. Symptoms:
- Sharp abdominal pain located underneath the right ribs.
- Usual morning sickness associated with pregnancy.
- Lower back pain experienced yesterday, which has since resolved.
- No bleeding or discharge.
- No concern for sexually transmitted diseases.
- No vomiting other than the usual morning sickness.
- No burning during urination.
- No fever, cough, congestion, runny nose, or chest pain.
- No recent contact with anyone who is sick.
- No vision changes or headaches.
- No other m

In [ ]:
#ros

In [231]:
summary_prompt = """
The following is a clinical conversation between a doctor and a patient. Generate a detailed Review Of Systems (ROS) for the patient. The ROS should include the following sections only:.

1. General: 
2. Skin: 
3. Head: 
4. Eyes: 
5. Ears, Nose, Throat: 
6. Cardiovascular: 
7. Respiratory: 
8. Gastrointestinal: 
9. Genitourinary: 
10. Musculoskeletal:
11. Neurological: 
12. Psychiatric:

Conversation Transcript:
{text}


Only include the above sections as mentioned in the transcript.Dont include additional details like physical eam findings or assessment plan.
"""

refine_prompt = """
Based on the provided clinical conversation and the following transcript, generate a detailed Review of Systems (RoS) for the patient. 
The RoS should cover the following 12 body systems and include specific details if mentioned, otherwise state 'denies any issues':

1. General: Any general symptoms such as fatigue, weight changes, or fever.
2. Skin: Any skin issues or rashes.
3. Head: Any headaches or head injuries.
4. Eyes: Vision problems or eye pain.
5. Ears, Nose, Throat: Hearing issues, nasal congestion, sore throat, etc.
6. Cardiovascular: Heart problems, chest pain, etc.
7. Respiratory: Breathing issues, cough, etc.
8. Gastrointestinal: Abdominal pain, nausea, etc.
9. Genitourinary: Urination problems, etc.
10. Musculoskeletal: Muscle pain, joint pain, etc.
11. Neurological: Numbness, tingling, etc.
12. Psychiatric: Depression, anxiety, etc.

Current RoS: {existing_summary}


Only include the above sections and no other information.
"""

summary_agent = SummarizerAgent(
    llm_model_name="gpt-4o",
    summary_prompt=summary_prompt,
    refine_prompt=refine_prompt,
    vector_store=None,
    kb_metadata=kb_metadata,
    document_location=None
)

with open('./meva/June12-01_AbdominalPain.json', 'r') as file:
    transcript_data = json.load(file)

summary_result = summary_agent.summarize(transcript_data['results']['transcripts'][0]['transcript'])
print(summary_result)

### Review of Systems (ROS)

1. **General:**
   - No fever.
   - No recent illness or contact with sick individuals.

2. **Skin:**
   - No symptoms reported.

3. **Head:**
   - No headaches.

4. **Eyes:**
   - No vision changes.

5. **Ears, Nose, Throat:**
   - No cough, congestion, or runny nose.
   - No sore throat.

6. **Cardiovascular:**
   - No chest pain.

7. **Respiratory:**
   - No cough.
   - No shortness of breath.

8. **Gastrointestinal:**
   - Sharp abdominal pain underneath right ribs.
   - Usual morning sickness, no vomiting beyond that.
   - No nausea reported apart from morning sickness.

9. **Genitourinary:**
   - No burning sensation during urination.
   - No bleeding or discharge.
   - No concern for STDs.

10. **Musculoskeletal:**
    - Lower back pain reported yesterday, now resolved.

11. **Neurological:**
    - No headaches.
    - No other neurological symptoms reported.

12. **Psychiatric:**
    - No psychiatric symptoms reported.


In [ ]:
#pe

In [255]:
summary_prompt = """
The following is a clinical conversation between a doctor and a patient. Generate a detailed list of the test names mentioned by the doctor in the Conversation Transcript. Refer to the refine prompt for more specific details.

Conversation Transcript:
{text}
"""

refine_prompt = """
Based on the provided clinical conversation and the following transcript, extract only the names of the tests mentioned by the doctor during the examination present in the Conversation Transcript. Do not include any medications or treatments, only list the tests.

Current Information: {existing_summary}

Please list only the test names mentioned by the doctor in the Conversation Transcript, excluding any medications or treatments. Only include the name of the tests and no other information.
"""

summary_agent = SummarizerAgent(
    llm_model_name="gpt-4o",
    summary_prompt=summary_prompt,
    refine_prompt=refine_prompt,
    vector_store=None,
    kb_metadata=kb_metadata,
    document_location=None
)

with open('./meva/June12-01_AbdominalPain.json', 'r') as file:
    transcript_data = json.load(file)

summary_result = summary_agent.summarize(transcript_data['results']['transcripts'][0]['transcript'])
print(summary_result)


1. Ultrasound
2. MRI
3. Blood test


In [ ]:
#Assessment Plan

In [224]:
summary_prompt = """
The following is a clinical conversation between a doctor and a patient. Generate a detailed Assessment Plan for the patient. The Assessment Plan should include the following sections only:.

1.Assessement:
2.Plan:

Conversation Transcript:
{text}

In the assessment section just mention abou tthe chief complaint alone and no other additional information.

"""

refine_prompt = """
"Based on the provided clinical conversation between the doctor and the patient, generate a detailed Assessment Plan. "
        "The Assessment Plan should include only the doctor's assessment and plan, clearly separated into two sections: Assessment and Plan. "
        "Each section should be detailed and follow the structure provided in the conversation.\n\n"
        "Assessment:\n"
        "Provide a short summary assessment of the patient's chief complaint based on the Conversation Transcript.Only mention about the chief complaint.\n\n"
        "Plan:\n"
        "1. List only the results of diagnostic tests given to the patient mentioned in Transcript Conversation.\n"
        "2. Include any relevant diagnostic results, such as EKG findings, if mentioned in the Transcript Conversation.\n"
        "3. Mention the diagnosis and follow-up instructions provided by the doctor in Transcript Conversation.\n"
        "Ensure not to include any medications or treatments that are not mentioned explicitly in the transcript."
        "Please ensure the assessment plan is comprehensive, clear, and formatted properly."
    )

Current Information: {existing_summary}


 Only include the name of the tests and no other information.
"""

summary_agent = SummarizerAgent(
    llm_model_name="gpt-4o",
    summary_prompt=summary_prompt,
    refine_prompt=refine_prompt,
    vector_store=None,
    kb_metadata=kb_metadata,
    document_location=None
)

with open('./meva/June12-01_AbdominalPain.json', 'r') as file:
    transcript_data = json.load(file)

summary_result = summary_agent.summarize(transcript_data['results']['transcripts'][0]['transcript'])
print(summary_result)

### Assessment:
- Chief Complaint: Sharp abdominal pain underneath the right ribs in a nine-week pregnant patient.

### Plan:
1. **Immediate Evaluation:**
   - Complete Blood Count (CBC)
   - Liver Function Tests (LFTs)
   - Urinalysis
   - Serum hCG level

2. **Imaging:**
   - Abdominal ultrasound

3. **Follow-Up:**
   - Referral to an obstetrician-gynecologist (OB-GYN)
   - Follow-up appointment within one week
